# Five ways to create CX2Network networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

In this tutorial you will learn to use CX2Network, a simple data model that is part of the ndex2 NDEx Client module.
CX2Network facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module version 3.8.0 or greater, 
[click here for installation instructions](https://ndex2.readthedocs.io/en/latest/installation.html) 



### Import Packages Required for this Tutorial

In [ ]:
from ndex2.cx2 import CX2Network
from ndex2.cx2 import RawCX2NetworkFactory
from ndex2.cx2 import NetworkXToCX2NetworkFactory
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import json

## Starting with an Empty Network

**Create an empty niceCx network**

In [ ]:
net_cx = CX2Network()

**Name the Network**

The _net_cx_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [ ]:
net_cx.set_name("Food Web")

**Add Nodes and Edges**

The block below creates a few nodes and a couple edges. Additional node/edge attributes can be added via the ``attributes`` named parameter

In [ ]:
fox_node = net_cx.add_node(attributes={'name': 'Fox'}, x=10, y=0)
mouse_node = net_cx.add_node(attributes={'name': 'Mouse'}, x=10, y=10)
bird_node = net_cx.add_node(attributes={'name': 'Bird'}, x=10, y=10)
fox_bird_edge = net_cx.add_edge(source=fox_node, target=bird_node,
                                attributes={'interaction': 'interacts-with'})

fox_mouse_edge = net_cx.add_edge(source=fox_node, target=mouse_node, 
                                 attributes={'interaction': 'interacts-with'})

print('The fox node has id: %d' % fox_node)

print('The mouse node has id: %d' % mouse_node)


The ``add_node()`` and ``add_edge()`` methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

 ``add_node_attribute()`` and ``add_edge_attribute()`` methods require the ID in the ``node_id`` named parameter along ``key`` set to the name of the attribute and ``value`` set to the value for that attribute.

In [ ]:
net_cx.add_node_attribute(node_id=fox_node, key='Color', value='Red')

net_cx.add_node_attribute(node_id=mouse_node, key='Color', value='Gray')

net_cx.add_node_attribute(node_id=bird_node, key='Color', value='Blue')


In [ ]:
net_cx.add_edge_attribute(edge_id=fox_mouse_edge, key='Hunted', value='On the ground')

We can now print _net_cx_ in CX2 format by utilizing ``to_cx2()`` method to get a list of dicts and ``json.dumps()`` 

In [ ]:
print(json.dumps(net_cx.to_cx2()))

## CX2 Files

In [ ]:
factory = RawCX2NetworkFactory()
net_cx_from_file = factory.get_cx2network('SimpleNetwork.cx2')

print(json.dumps(net_cx_from_file.to_cx2()))

## NDEx Networks

In [ ]:
# Create NDEx2 python client
client = ndex2.client.Ndex2()

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Download CoCaNet2 from NDEx
# https://www.ndexbio.org/viewer/networks/f1dd6cc3-0007-11e6-b550-06603eb7f303
client_resp = client.get_network_as_cx2_stream('f1dd6cc3-0007-11e6-b550-06603eb7f303')

# Convert downloaded network to CX2Network object
net_cx_from_ndex = factory.get_cx2network(json.loads(client_resp.content))


# Display information about network and output 1st 100 characters of CX2
print('Name: ' + net_cx_from_ndex.get_name())
print('Number of nodes: ' + str(len(net_cx_from_ndex.get_nodes())))
print('Number of nodes: ' + str(len(net_cx_from_ndex.get_edges())))
print(json.dumps(net_cx_from_ndex.to_cx2(), indent=2)[0:100])

## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [ ]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [ ]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, CXNetwork objects are created from both *G* and *path_subgraph*.

In [ ]:
# Creating an instance of NetworkXToCX2NetworkFactory
factory = NetworkXToCX2NetworkFactory()

G.name = 'Created from NetworkX (full)'
net_cx_full = factory.get_cx2network(G)

path_subgraph.name = 'Created from NetworkX (shortest path)'
net_cx_short = factory.get_cx2network(path_subgraph)
                       

print(G.edges())
print('')
print('Name: ' + net_cx_full.get_name())
print('Number of nodes: ' + str(len(net_cx_full.get_nodes())))
print('Number of nodes: ' + str(len(net_cx_full.get_edges())))

print(path_subgraph.edges())
print('')
print('Name: ' + net_cx_short.get_name())
print('Number of nodes: ' + str(len(net_cx_short.get_nodes())))
print('Number of nodes: ' + str(len(net_cx_short.get_edges())))


## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

nice_cx_df_2_column = ndex2.create_nice_cx_from_pandas(df)

nice_cx_df_2_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_2_col_from_niceCx = nice_cx_df_2_column.to_pandas_dataframe()
print(df_2_col_from_niceCx)

### 3 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

nice_cx_df_3_column = ndex2.create_nice_cx_from_pandas(df)

nice_cx_df_3_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_3_col_from_niceCx = nice_cx_df_3_column.to_pandas_dataframe()
print(df_3_col_from_niceCx)

### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [ ]:
df = pd.DataFrame.from_dict(dict([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

nice_cx_df_with_headers = ndex2.create_nice_cx_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

nice_cx_df_with_headers.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_headers_from_niceCx = nice_cx_df_with_headers.to_pandas_dataframe()
print(df_headers_from_niceCx)

### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [ ]:
#======================
# CONVERT TO DATAFRAME
#======================
nice_cx_from_server_df = nice_cx_from_server.to_pandas_dataframe()
print(nice_cx_from_server_df)

Likewise, networks can be output as a networkx graph.

In [ ]:
#=====================
# CONVERT TO NETWORKX
#=====================
nice_cx_from_server_to_netx = nice_cx_from_server.to_networkx(mode='default')

print(nice_cx_from_server_to_netx)

**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [ ]:
my_account="enter your username here"
my_password="enter your password here"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

NiceCx networks can be saved to the NDEx server by calling **upload_to()**

In [ ]:
if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    upload_message = nice_cx_df_with_headers.upload_to(my_server, my_account, my_password)
    print(upload_message)